# Tile Generation Tutorial (File Edition)

Welcome to the tile generation tutorial!

As a whole slide image is too large for deep learning model training, a slide is often divded into a set of small tiles, and used for training. For tile-based whole slide image analysis, generating tiles and labels is an important and laborious step. With LUNA tiling CLIs and tutorials, you can easily generate tile labels and get your data ready for downstream analysis. In this notebook, we will see how to generate tiles and labels using LUNA tiling CLIs. Here are the main steps we will review:

1. Load slides
2. Generate tiles, labels
3. Collect tiles for model training

Through out this notebook, we will use different method parameter files. Please refer to the example parameter files in the `configs` directory to follow these steps.


In [1]:
import os
HOME = os.environ['HOME']

In [2]:
env DATASET_URL=file:///$HOME/vmount/PRO-12-123/

env: DATASET_URL=file:////home/pashaa/vmount/PRO-12-123/


Initially, we'll walk through each CLI step manually-- then run them using the LunaCLIClient in parallel

First, we generate tiles given a slide image of size 128 at 20x, and save them

In [3]:
!generate_tiles \
file:~/vmount/PRO-12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs \
--tile_size 128 --requested_magnification 10 \
-o ~/vmount/PRO-12-123/tiling/test/tiles


2022-08-22 11:21:41,608 - INFO - root - Initalized logger, log file at: luna.log
2022-08-22 11:21:44,867 - INFO - luna.common.utils - Started CLI Runner wtih <function generate_tiles at 0x7f259f988dc0>
2022-08-22 11:21:44,869 - INFO - luna.common.utils - Validating params...
2022-08-22 11:21:44,872 - INFO - luna.common.utils -  -> Set input_slide_image (<class 'str'>) = file:~/vmount/PRO-12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs
2022-08-22 11:21:44,873 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = /home/pashaa/vmount/PRO-12-123/tiling/test/tiles
2022-08-22 11:21:44,876 - INFO - luna.common.utils -  -> Set tile_size (<class 'int'>) = 128
2022-08-22 11:21:44,877 - INFO - luna.common.utils -  -> Set requested_magnification (<class 'float'>) = 10.0
2022-08-22 11:21:44,894 - INFO - luna.common.utils - Expanding inputs...
2022-08-22 11:21:44,896 - INFO - luna.common.utils - Attempting to read metadata at file:~/vmount/PRO-12-123/data/toy_data_set

In [4]:
!detect_tissue \
~/vmount/PRO-12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs \
~/vmount/PRO-12-123/tiling/test/tiles \
--requested_magnification 2 \
--filter_query "otsu_score > 0.1" \
-o ~/vmount/PRO-12-123/tiling/test/detect

2022-08-22 11:23:56,325 - INFO - root - Initalized logger, log file at: luna.log
2022-08-22 11:23:58,171 - INFO - luna.common.utils - Started CLI Runner wtih <function detect_tissue at 0x7f4e09297310>
2022-08-22 11:23:58,173 - INFO - luna.common.utils - Validating params...
2022-08-22 11:23:58,174 - INFO - luna.common.utils -  -> Set input_slide_image (<class 'str'>) = /home/pashaa/vmount/PRO-12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs
2022-08-22 11:23:58,175 - INFO - luna.common.utils -  -> Set input_slide_tiles (<class 'str'>) = /home/pashaa/vmount/PRO-12-123/tiling/test/tiles
2022-08-22 11:23:58,177 - INFO - luna.common.utils -  -> Set requested_magnification (<class 'float'>) = 2.0
2022-08-22 11:23:58,178 - INFO - luna.common.utils -  -> Set filter_query (<class 'str'>) = otsu_score > 0.1
2022-08-22 11:23:58,180 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = /home/pashaa/vmount/PRO-12-123/tiling/test/detect
2022-08-22 11:23:58,181 - INFO -

In [5]:
!label_tiles \
../PRO-12-123/data/toy_data_set/table/ANNOTATIONS ~/vmount/PRO-12-123/tiling/test/detect \
-o ~/vmount/PRO-12-123/tiling/test/label

2022-08-22 11:42:37,912 - INFO - root - Initalized logger, log file at: luna.log
2022-08-22 11:42:37,916 - INFO - luna.common.utils - Started CLI Runner wtih <function generate_tile_labels at 0x7fe4784541f0>
2022-08-22 11:42:37,918 - INFO - luna.common.utils - Validating params...
2022-08-22 11:42:37,919 - INFO - luna.common.utils -  -> Set input_slide_annotation_dataset (<class 'str'>) = ../PRO-12-123/data/toy_data_set/table/ANNOTATIONS
2022-08-22 11:42:37,920 - INFO - luna.common.utils -  -> Set input_slide_tiles (<class 'str'>) = /home/pashaa/vmount/PRO-12-123/tiling/test/detect
2022-08-22 11:42:37,922 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = /home/pashaa/vmount/PRO-12-123/tiling/test/label
2022-08-22 11:42:37,925 - INFO - luna.common.utils - Expanding inputs...
2022-08-22 11:42:37,927 - INFO - luna.common.utils - Attempting to read metadata at ../PRO-12-123/data/toy_data_set/table/ANNOTATIONS/metadata.yml
2022-08-22 11:42:37,931 - INFO - luna.common.utils -

In [6]:
!save_tiles \
~/vmount/PRO-12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs \
~/vmount/PRO-12-123/tiling/test/label \
--num_cores 4 --batch_size 200 --dataset_id PRO_TILES \
-o ~/vmount/PRO-12-123/tiling/test/saved_tiles

2022-08-22 11:42:55,310 - INFO - root - Initalized logger, log file at: luna.log
2022-08-22 11:42:57,188 - INFO - luna.common.utils - Started CLI Runner wtih <function save_tiles at 0x7f5279659dc0>
2022-08-22 11:42:57,190 - INFO - luna.common.utils - Validating params...
2022-08-22 11:42:57,191 - INFO - luna.common.utils -  -> Set input_slide_image (<class 'str'>) = /home/pashaa/vmount/PRO-12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs
2022-08-22 11:42:57,192 - INFO - luna.common.utils -  -> Set input_slide_tiles (<class 'str'>) = /home/pashaa/vmount/PRO-12-123/tiling/test/label
2022-08-22 11:42:57,193 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = /home/pashaa/vmount/PRO-12-123/tiling/test/saved_tiles
2022-08-22 11:42:57,196 - INFO - luna.common.utils -  -> Set num_cores (<class 'int'>) = 4
2022-08-22 11:42:57,197 - INFO - luna.common.utils -  -> Set batch_size (<class 'int'>) = 200
2022-08-22 11:42:57,201 - INFO - luna.common.utils - Expanding 

In [7]:
from luna.common.utils import LunaCliClient

def pipeline (slide_id, input_slide, input_annotations):
    client = LunaCliClient("~/vmount/PRO-12-123/2_tiling-file", slide_id)
    
    client.bootstrap("slide", input_slide)
    client.bootstrap("annotations", input_annotations)
    
    client.configure("generate_tiles", "slide", 
        tile_size=128, 
        requested_magnification=10
    ).run("source_tiles")

    client.configure("detect_tissue", "slide", "source_tiles",
        filter_query="otsu_score > 0.1", 
        requested_magnification=2
    ).run("detected_tiles")

    client.configure("label_tiles", "annotations", "detected_tiles").run("labled_tiles")

    client.configure( "save_tiles", "slide", "labled_tiles",
        num_cores=4, batch_size=200, dataset_id='PRO_TILES_LABELED'
    ).run("saved_tiles")

In [8]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

df_slides = pd.read_parquet("../PRO-12-123/data/toy_data_set/table/SLIDES/slide_ingest_PRO-12-123.parquet")
        
with ThreadPoolExecutor(5) as pool:
    
    for index, row in df_slides.iterrows():
        print (index)
        
        pool.submit(pipeline, index, row.slide_image, "../PRO-12-123/data/toy_data_set/table/ANNOTATIONS")
        

01OV008-308ad404-7079-4ff8-8232-12ee2e
generate_tiles file:/home/pashaa/vmount/PRO-12-123/data/toy_data_set/01OV008-308ad404-7079-4ff8-8232-12ee2e.svs --tile_size 128 --requested_magnification 10
['generate_tiles', PosixPath('file:/home/pashaa/vmount/PRO-12-123/data/toy_data_set/01OV008-308ad404-7079-4ff8-8232-12ee2e.svs'), '--tile_size', '128', '--requested_magnification', '10', '-o', '/home/pashaa/vmount/PRO-12-123/2_tiling-file/01OV008-308ad404-7079-4ff8-8232-12ee2e/source_tiles']
01OV002-ed65cf94-8bc6-492b-9149-adc16f
generate_tiles file:/home/pashaa/vmount/PRO-12-123/data/toy_data_set/01OV002-ed65cf94-8bc6-492b-9149-adc16f.svs --tile_size 128 --requested_magnification 10
['generate_tiles', PosixPath('file:/home/pashaa/vmount/PRO-12-123/data/toy_data_set/01OV002-ed65cf94-8bc6-492b-9149-adc16f.svs'), '--tile_size', '128', '--requested_magnification', '10', '-o', '/home/pashaa/vmount/PRO-12-123/2_tiling-file/01OV002-ed65cf94-8bc6-492b-9149-adc16f/source_tiles']
01OV002-bd8cdc70-3d46-

2022-08-22 11:46:29,576 - INFO - root - Initalized logger, log file at: luna.log
2022-08-22 11:46:29,582 - INFO - root - Initalized logger, log file at: luna.log
2022-08-22 11:46:29,600 - INFO - root - Initalized logger, log file at: luna.log
2022-08-22 11:46:29,630 - INFO - root - Initalized logger, log file at: luna.log
2022-08-22 11:46:29,651 - INFO - root - Initalized logger, log file at: luna.log
2022-08-22 11:46:31,759 - INFO - luna.common.utils - Started CLI Runner wtih <function generate_tiles at 0x7f72db216d30>
2022-08-22 11:46:31,761 - INFO - luna.common.utils - Validating params...
2022-08-22 11:46:31,763 - INFO - luna.common.utils -  -> Set input_slide_image (<class 'str'>) = file:/home/pashaa/vmount/PRO-12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs
2022-08-22 11:46:31,765 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = /home/pashaa/vmount/PRO-12-123/2_tiling-file/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77/source_tiles
2022-08-22 11:46:31

2022-08-22 11:46:32,127 - INFO - generate_tiles - Slide size = [53760,54840]
2022-08-22 11:46:32,131 - INFO - generate_tiles - Normalized magnification scale factor for 10.0x is 4.0
2022-08-22 11:46:32,133 - INFO - generate_tiles - Requested tile size=128, tile size at full magnficiation=512
2022-08-22 11:46:32,138 - INFO - generate_tiles - tiles x 105, tiles y 108
2022-08-22 11:46:32,153 - INFO - generate_tiles - Slide size = [50880,49787]
2022-08-22 11:46:32,155 - INFO - generate_tiles - Normalized magnification scale factor for 10.0x is 4.0
2022-08-22 11:46:32,157 - INFO - generate_tiles - Requested tile size=128, tile size at full magnficiation=512
2022-08-22 11:46:32,160 - INFO - generate_tiles - tiles x 100, tiles y 98
2022-08-22 11:46:32,167 - INFO - generate_tiles - Number of tiles in raster: 10918
2022-08-22 11:46:32,182 - INFO - generate_tiles - Number of tiles in raster: 9408
2022-08-22 11:46:32,207 - INFO - generate_tiles -                x_coord  y_coord  xy_extent  tile_s

2022-08-22 11:46:32,637 - INFO - luna.common.utils - Code block 'transform::generate_tiles' took: 0.7036285760000283s
2022-08-22 11:46:32,640 - INFO - luna.common.utils - ------------------------------------------------------------
2022-08-22 11:46:32,640 - INFO - luna.common.utils -  Done with transform, running post-transform functions... 
2022-08-22 11:46:32,640 - INFO - luna.common.utils - ------------------------------------------------------------
2022-08-22 11:46:32,649 - INFO - luna.common.utils - Done.


detect_tissue file:/home/pashaa/vmount/PRO-12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs /home/pashaa/vmount/PRO-12-123/2_tiling-file/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77/source_tiles --filter_query otsu_score > 0.1 --requested_magnification 2
['detect_tissue', PosixPath('file:/home/pashaa/vmount/PRO-12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs'), '/home/pashaa/vmount/PRO-12-123/2_tiling-file/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77/source_tiles', '--filter_query', 'otsu_score > 0.1', '--requested_magnification', '2', '-o', '/home/pashaa/vmount/PRO-12-123/2_tiling-file/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77/detected_tiles']
detect_tissue file:/home/pashaa/vmount/PRO-12-123/data/toy_data_set/01OV002-ed65cf94-8bc6-492b-9149-adc16f.svs /home/pashaa/vmount/PRO-12-123/2_tiling-file/01OV002-ed65cf94-8bc6-492b-9149-adc16f/source_tiles --filter_query otsu_score > 0.1 --requested_magnification 2
['detect_tissue', PosixPath('file:/home/pashaa/vmount/P

...

100%|██████████| 5317/5317 [00:02<00:00, 1975.33it/s]
2022-08-22 12:04:53,587 - INFO - generate_tile_labels -                 x_coord  y_coord  ...  regional_label  intersection_area
2022-08-22 12:04:53,587 - INFO - generate_tile_labels - address                           ...                                   
2022-08-22 12:04:53,587 - INFO - generate_tile_labels - x12_y28_z10.0      6144    14336  ...             fat           0.442006
2022-08-22 12:04:53,587 - INFO - generate_tile_labels - x12_y29_z10.0      6144    14848  ...             fat           0.761986
2022-08-22 12:04:53,587 - INFO - generate_tile_labels - x12_y30_z10.0      6144    15360  ...             fat           0.272937
2022-08-22 12:04:53,587 - INFO - generate_tile_labels - x13_y27_z10.0      6656    13824  ...             fat           0.147144
2022-08-22 12:04:53,587 - INFO - generate_tile_labels - x13_y28_z10.0      6656    14336  ...             fat           0.992228
2022-08-22 12:04:53,587 - INFO - generate_t

save_tiles file:/home/pashaa/vmount/PRO-12-123/data/toy_data_set/01OV007-9b90eb78-2f50-4aeb-b010-d642f9.svs /home/pashaa/vmount/PRO-12-123/2_tiling-file/01OV007-9b90eb78-2f50-4aeb-b010-d642f9/labled_tiles --num_cores 4 --batch_size 200 --dataset_id PRO_TILES_LABELED
['save_tiles', PosixPath('file:/home/pashaa/vmount/PRO-12-123/data/toy_data_set/01OV007-9b90eb78-2f50-4aeb-b010-d642f9.svs'), '/home/pashaa/vmount/PRO-12-123/2_tiling-file/01OV007-9b90eb78-2f50-4aeb-b010-d642f9/labled_tiles', '--num_cores', '4', '--batch_size', '200', '--dataset_id', 'PRO_TILES_LABELED', '-o', '/home/pashaa/vmount/PRO-12-123/2_tiling-file/01OV007-9b90eb78-2f50-4aeb-b010-d642f9/saved_tiles']


2022-08-22 12:04:55,506 - INFO - root - Initalized logger, log file at: luna.log
2022-08-22 12:04:59,558 - INFO - luna.common.utils - Started CLI Runner wtih <function save_tiles at 0x7fcc314aedc0>
2022-08-22 12:04:59,561 - INFO - luna.common.utils - Validating params...
2022-08-22 12:04:59,563 - INFO - luna.common.utils -  -> Set input_slide_image (<class 'str'>) = file:/home/pashaa/vmount/PRO-12-123/data/toy_data_set/01OV007-9b90eb78-2f50-4aeb-b010-d642f9.svs
2022-08-22 12:04:59,565 - INFO - luna.common.utils -  -> Set input_slide_tiles (<class 'str'>) = /home/pashaa/vmount/PRO-12-123/2_tiling-file/01OV007-9b90eb78-2f50-4aeb-b010-d642f9/labled_tiles
2022-08-22 12:04:59,566 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = /home/pashaa/vmount/PRO-12-123/2_tiling-file/01OV007-9b90eb78-2f50-4aeb-b010-d642f9/saved_tiles
2022-08-22 12:04:59,568 - INFO - luna.common.utils -  -> Set num_cores (<class 'int'>) = 4
2022-08-22 12:04:59,570 - INFO - luna.common.utils -  -> Set ba

100%|██████████| 27/27 [01:22<00:00,  3.05s/it]

2022-08-22 12:06:24,272 - INFO - generate_tiles -                 x_coord  ...                                         tile_store
2022-08-22 12:06:24,272 - INFO - generate_tiles - address                  ...                                                   
2022-08-22 12:06:24,272 - INFO - generate_tiles - x3_y74_z10.0       1536  ...  /home/pashaa/vmount/PRO-12-123/2_tiling-file/0...
2022-08-22 12:06:24,272 - INFO - generate_tiles - x3_y75_z10.0       1536  ...  /home/pashaa/vmount/PRO-12-123/2_tiling-file/0...
2022-08-22 12:06:24,272 - INFO - generate_tiles - x3_y76_z10.0       1536  ...  /home/pashaa/vmount/PRO-12-123/2_tiling-file/0...
2022-08-22 12:06:24,272 - INFO - generate_tiles - x3_y77_z10.0       1536  ...  /home/pashaa/vmount/PRO-12-123/2_tiling-file/0...
2022-08-22 12:06:24,272 - INFO - generate_tiles - x3_y78_z10.0       1536  ...  /home/pashaa/vmount/PRO-12-123/2_tiling-file/0...
2022-08-22 12:06:24,272 - INFO - generate_tiles - ...                 ...  ...            

In [9]:
import pandas as pd
df_tiles = pd.read_parquet("~/vmount/PRO-12-123/datasets/PRO_TILES_LABELED/").query("intersection_area > 0")
print (df_tiles['regional_label'].value_counts())
df_tiles

tumor     605
fat       277
stroma    263
Name: regional_label, dtype: int64


,SEGMENT_ID,dsa_collection_uuid,slide_id,address,x_coord,y_coord,xy_extent,tile_size,tile_units,otsu_score,regional_label,intersection_area,tile_store
1163,63035f7a18325e4c6e929591-01OV002-bd8cdc70-3d46...,63035f7a18325e4c6e929591,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,x26_y56_z10.0,13312,28672,512,128,px,0.88,stroma,0.053094,/home/pashaa/vmount/PRO-12-123/2_tiling-file/0...
1164,63035f7a18325e4c6e929591-01OV002-bd8cdc70-3d46...,63035f7a18325e4c6e929591,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,x26_y57_z10.0,13312,29184,512,128,px,0.63,stroma,0.341454,/home/pashaa/vmount/PRO-12-123/2_tiling-file/0...
1223,63035f7a18325e4c6e929591-01OV002-bd8cdc70-3d46...,63035f7a18325e4c6e929591,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,x27_y56_z10.0,13824,28672,512,128,px,0.76,stroma,0.655530,/home/pashaa/vmount/PRO-12-123/2_tiling-file/0...
1224,63035f7a18325e4c6e929591-01OV002-bd8cdc70-3d46...,63035f7a18325e4c6e929591,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,x27_y57_z10.0,13824,29184,512,128,px,0.58,stroma,0.898266,/home/pashaa/vmount/PRO-12-123/2_tiling-file/0...
1277,63035f7a18325e4c6e929591-01OV002-bd8cdc70-3d46...,63035f7a18325e4c6e929591,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,x28_y55_z10.0,14336,28160,512,128,px,0.84,stroma,0.258913,/home/pashaa/vmount/PRO-12-123/2_tiling-file/0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22454,63035f7a18325e4c6e929591-01OV008-7579323e-2fae...,63035f7a18325e4c6e929591,01OV008-7579323e-2fae-43a9-b00f-a15c28,x54_y41_z10.0,27648,20992,512,128,px,0.64,stroma,0.528478,/home/pashaa/vmount/PRO-12-123/2_tiling-file/0...
22507,63035f7a18325e4c6e929591-01OV008-7579323e-2fae...,63035f7a18325e4c6e929591,01OV008-7579323e-2fae-43a9-b00f-a15c28,x55_y39_z10.0,28160,19968,512,128,px,0.38,stroma,0.465030,/home/pashaa/vmount/PRO-12-123/2_tiling-file/0...
22508,63035f7a18325e4c6e929591-01OV008-7579323e-2fae...,63035f7a18325e4c6e929591,01OV008-7579323e-2fae-43a9-b00f-a15c28,x55_y40_z10.0,28160,20480,512,128,px,0.53,stroma,0.893236,/home/pashaa/vmount/PRO-12-123/2_tiling-file/0...
22509,63035f7a18325e4c6e929591-01OV008-7579323e-2fae...,63035f7a18325e4c6e929591,01OV008-7579323e-2fae-43a9-b00f-a15c28,x55_y41_z10.0,28160,20992,512,128,px,0.73,stroma,0.034031,/home/pashaa/vmount/PRO-12-123/2_tiling-file/0...


Congratulations! Now you have 2120 tumor, 860 stroma, and 751 fat tiles images and labels ready to train your model.